## Install Packages and Importing Libraries

In [20]:
# pip install numpy librosa torch torchaudio

In [21]:
# General Imports
import os
import wave
import numpy as np
from sklearn.preprocessing import LabelEncoder
import random
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import librosa
import torch
import torch.nn as nn
import torch.optim as optim

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


## Deleteing unneeded folders

In [22]:
mainFolderLocation = "ABI-1 Corpus/accents/"
listOfAccents = os.listdir(mainFolderLocation)

# Goes through list of accent folders
for accent in listOfAccents:
    accentFolderLocation = mainFolderLocation + accent + "/"
    if accent == ".DS_Store":
        os.remove(accentFolderLocation[:-1])
        break

    accentContent = os.listdir(accentFolderLocation)

    # Goes through list of gender folders
    for genders in accentContent:
        genderFolderLocation = accentFolderLocation + genders + "/"
        if genders == ".DS_Store":
          os.remove(genderFolderLocation[:-1])
          break

        genderContent = os.listdir(genderFolderLocation[:-1])
        
        # Goes through list of participant folders
        for participant in genderContent:
            participantFolderLocation = genderFolderLocation + participant + "/"
            if participant == ".DS_Store":
                os.remove(participantFolderLocation[:-1])
                break

            participantContent = os.listdir(participantFolderLocation)
            print(participantContent)

            for file in participantContent:
                fileAddress = participantFolderLocation + file
                if file[-3:] != "wav":
                    os.remove(fileAddress)
                    break

                if file[:12] != "shortpassage":
                    os.remove(fileAddress)
                    break


['cwd_ann.trs', 'cwd_ann.txt', 'cwd_CT.wav', 'cwe_ann.trs', 'cwe_ann.txt', 'cwe_CT.wav', 'shortpassagea_ann.trs', 'shortpassagea_ann.txt', 'shortpassagea_CT.wav', 'shortpassageb_ann.trs', 'shortpassageb_ann.txt', 'shortpassageb_CT.wav', 'shortpassagec_ann.trs', 'shortpassagec_ann.txt', 'shortpassagec_CT.wav', 'shortphrases_ann.trs', 'shortphrases_ann.txt', 'shortphrases_CT.wav', 'shortsentences_ann.trs', 'shortsentences_ann.txt', 'shortsentences_CT.wav']
['cwd_ann.trs', 'cwd_ann.txt', 'cwd_CT.wav', 'cwe_ann.trs', 'cwe_ann.txt', 'cwe_CT.wav', 'shortpassagea_ann.trs', 'shortpassagea_ann.txt', 'shortpassagea_CT.wav', 'shortpassageb_ann.trs', 'shortpassageb_ann.txt', 'shortpassageb_CT.wav', 'shortpassagec_ann.trs', 'shortpassagec_ann.txt', 'shortpassagec_CT.wav', 'shortphrases_ann.trs', 'shortphrases_ann.txt', 'shortphrases_CT.wav', 'shortsentences_ann.trs', 'shortsentences_ann.txt', 'shortsentences_CT.wav']
['cwd_ann.trs', 'cwd_ann.txt', 'cwd_CT.wav', 'cwe_ann.trs', 'cwe_ann.txt', 'cwe_CT

## Obtaining list of directories

In [23]:
def getSpeakerRoots(datapath = "ABI-1 Corpus\\accents"):
    speakerList = []
    accentSubfolder = [f.path for f in os.scandir(datapath) if f.is_dir()]
    for accent in accentSubfolder:
        for gender in ["female", "male"]:
            speakerFolders = os.listdir(os.path.join(accent, gender))
            for speaker in speakerFolders:
                if not speaker.startswith("."):
                    speakerList.append(os.path.join(accent, gender, speaker))
        
    return speakerList

In [24]:
speakerList = getSpeakerRoots()

## Getting .wav files

In [25]:
def getWavFile(datapath):
    files = os.listdir(datapath)
    filesWav = [i for i in files if i.endswith(".wav")]
    return filesWav

In [26]:
speakers = []
for i in speakerList:
    speakers.append(i[i.rfind("\\")+1:])

speakerFilePairs = {}
for i in range(len(speakers)):
    speakerFilePairs[speakers[i]] = [getWavFile(speakerList[i]), speakerList[i]]

## Segmenting the data

In [27]:
folderName = "SegmentedDataset"
if os.path.exists(folderName) == False:
    os.mkdir(folderName)

for speaker in list(speakerFilePairs.keys()):
    speakerFolder = folderName + "\\" + speaker
    if os.path.exists(speakerFolder) == False:
        os.mkdir(speakerFolder)

In [28]:
def splitWavFile(inputFile, seconds):
    for recording in inputFile[0]:
        fileLocation = inputFile[1] + "\\" + recording
        with wave.open(fileLocation, "rb") as wavFile:
            params = wavFile.getparams()
            sampleRate = params.framerate
            totalFrames = params.nframes

            # Calculate the number of frames per segment
            framesPerSegment = seconds * sampleRate
            totalSegment = totalFrames // framesPerSegment
            
            for segmentIndex in range(totalSegment):
                startFrame = segmentIndex * framesPerSegment
                wavFile.setpos(startFrame)
                audioData = np.frombuffer(wavFile.readframes(framesPerSegment), dtype=np.int16)
                fileName = inputFile[1][inputFile[1].rfind("\\")+1:]
                outputFile = "SegmentedDataset\\" + fileName + "\\chunk" + str(len(os.listdir("SegmentedDataset\\" + fileName))) + ".wav"

                with wave.open(outputFile, "wb") as outputWav:
                    outputWav.setparams(params)
                    outputWav.writeframes(audioData.tobytes())

            startFrame = totalSegment*framesPerSegment
            wavFile.setpos(startFrame)
            audioData = np.frombuffer(wavFile.readframes(framesPerSegment), dtype=np.int16)
            outputFile = "SegmentedDataset\\" + fileName + "\\chunk" + str(len(os.listdir("SegmentedDataset\\" + fileName))) + ".wav"
            
            with wave.open(outputFile, "wb") as outputWav:
                outputWav.setparams(params)
                outputWav.writeframes(audioData.tobytes())

In [29]:
# rebootFiles = True
rebootFiles = False
if len(os.listdir("SegmentedDataset")[-1]) == 0 or rebootFiles == True:
    indexes = list(speakerFilePairs.keys())

    for index in indexes:
        folderAddress = "SegmentedDataset\\" + index + "\\"
        for file in os.listdir(folderAddress):
            os.remove(folderAddress + file)
    
    for index in indexes:
        splitWavFile(speakerFilePairs[index], 3)

## Getting each data point

In [30]:
dataLocations = []
for speaker in os.listdir(folderName):
    for file in os.listdir(folderName + "\\" + speaker):
        dataLocations.append((speaker, folderName + "\\" + speaker + "\\" + file))

## Splitting Data

In [31]:
labels = LabelEncoder()
(labels.fit(speakers))

dataLocationsWithLabels = []
for entry in dataLocations:
    dataLocationsWithLabels.append((labels.transform([entry[0]])[0], entry[1]))

In [32]:
random.shuffle(dataLocationsWithLabels)
tempTrainSet = dataLocationsWithLabels[:int(len(dataLocationsWithLabels)*0.8)]

trainSet = tempTrainSet[:int(len(tempTrainSet)*0.8)]
testSet = dataLocationsWithLabels[int(len(dataLocationsWithLabels)*0.8):]
validationSet = tempTrainSet[int(len(tempTrainSet)*0.8):]

## Creating Dataset Class

In [33]:
class speechDataset(Dataset):
    def __init__(self, inputData):
        self.data = inputData

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        label, filePath = self.data[index]
        audio, sampleRate = librosa.load(filePath)
        mfcc = librosa.feature.mfcc(y=audio, sr=sampleRate, n_mfcc=13)

        maxLength = 130
        if mfcc.shape[1] < maxLength:
            padWidth = maxLength - mfcc.shape[1]
            mfcc = np.pad(mfcc, pad_width=((0,0),(0, padWidth)), mode="constant")

        mfccTensor = torch.tensor(mfcc).unsqueeze(0)
        return mfccTensor, label

In [34]:
trainSetObject = speechDataset(trainSet)
testSetObject = speechDataset(testSet)
validationSetObject = speechDataset(validationSet)

batchSize = 64
trainDataloader = DataLoader(trainSetObject, batch_size=batchSize)
testDataloader = DataLoader(testSetObject, batch_size=batchSize)
validationDataloader = DataLoader(validationSetObject, batch_size=batchSize)

# Old

## Creating the model

In [35]:
class speechModel(nn.Module):
    def __init__(self, inputShape, numberOfSpeakers):
        super(speechModel, self).__init__()

        # Convolution Blocks
        self.cnnLayers = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
        )

        # LSTM-RNN
        cnnOutputShape = self.getCNNOutputShape(inputShape)
        self.lstm = nn.LSTM(input_size=(cnnOutputShape[1]*cnnOutputShape[2]*cnnOutputShape[3]), hidden_size=128)

        # Fully Connected Layer
        self.fc = nn.Linear(in_features=128, out_features=numberOfSpeakers)

    def forward(self, x):
        # CNN Layer
        x = self.cnnLayers(x)
        x = x.view(x.size(0), -1)

        # LSTM layer
        x, _ = self.lstm(x.unsqueeze(1))
        x = x[:, -1, :]

        # Fully connected layers
        x = self.fc(x)

        return x

    def getCNNOutputShape(self, input_shape):
        input_tensor = torch.randn(1, *input_shape)
        with torch.no_grad():
            output_tensor = self.cnnLayers(input_tensor)
        return output_tensor.shape

In [36]:
# Testing Loop
learningRate = 0.001
numberOfEpochs = 10

inputShape = (1, 13, 130)
numberOfSpeakers = len(speakers)

model = speechModel(inputShape, numberOfSpeakers)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learningRate)

In [38]:
for epoch in range(numberOfEpochs):
    total_loss = 0.0

    for inputs, labels in testDataloader:  
        # Zero the gradients
        optimizer.zero_grad()

         # Forward pass
        outputs = model(inputs)

        # Compute the loss
        loss = criterion(outputs, labels.to(torch.long))

        # Backward pass
        loss.backward()

        # Update the parameters
        optimizer.step()

        # Accumulate the total loss for monitoring
        total_loss += loss.item()
    
    average_loss = total_loss / len(trainDataloader)  # Adjust based on your dataloader
    print(f'Epoch {epoch + 1}/{numberOfEpochs}, Loss: {average_loss}')
    break


Epoch 1/10, Loss: 1.7959492844595035
